# <전처리>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/금융빅데이터분석가/신용카드 연체 예측

/content/drive/MyDrive/금융빅데이터분석가/신용카드 연체 예측


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import seaborn as sns

In [ ]:
df = pd.read_csv("data/train.csv")
df.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

In [ ]:
df.describe()

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit
count,26457.000000,26457.000000,2.645700e+04,26457.000000,26457.000000,26457.0,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000
mean,13228.000000,0.428658,1.873065e+05,-15958.053899,59068.750728,1.0,0.224742,0.294251,0.091280,2.196848,-26.123294,1.519560
std,7637.622372,0.747326,1.018784e+05,4201.589022,137475.427503,0.0,0.417420,0.455714,0.288013,0.916717,16.559550,0.702283
min,0.000000,0.000000,2.700000e+04,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,1.000000,-60.000000,0.000000
25%,6614.000000,0.000000,1.215000e+05,-19431.000000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.000000,-39.000000,1.000000
50%,13228.000000,0.000000,1.575000e+05,-15547.000000,-1539.000000,1.0,0.000000,0.000000,0.000000,2.000000,-24.000000,2.000000
75%,19842.000000,1.000000,2.250000e+05,-12446.000000,-407.000000,1.0,0.000000,1.000000,0.000000,3.000000,-12.000000,2.000000
max,26456.000000,19.000000,1.575000e+06,-7705.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.000000,0.000000,2.000000


In [ ]:
df.shape

(26457, 20)

## 1) 중복 데이터 확인

- 'begin_month', 'credit' 변수를 제외한 중복 데이터 10,000개 이상  
=> 동일 인물이 다른 시기에 카드 개설했다고 가정  
=> 데이터의 절반이 중복 데이터?  
=> 한 행으로 줄이는 대신 '개설 카드 개수' 열 추가

- 동일 인물이 카드를 개설하는 시기에 따라 신용도가 다를 수 있으므로 가장 최신의 카드를 개설했을 당시의 신용도를 남겨두고 나머지 데이터를 버림

In [ ]:
df.columns

Index(['index', 'gender', 'car', 'reality', 'child_num', 'income_total',
       'income_type', 'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_MOBIL', 'work_phone', 'phone', 'email',
       'occyp_type', 'family_size', 'begin_month', 'credit'],
      dtype='object')

In [ ]:
cols = ['gender', 'car', 'reality', 'child_num', 'income_total',
       'income_type', 'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_MOBIL', 'work_phone', 'phone', 'email',
       'occyp_type', 'family_size', 'credit']
df[df.duplicated(cols)]

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
319,319,F,N,N,1,225000.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-12640,-399,1,0,0,0,Accountants,3.0,-21.0,2.0
330,330,F,N,N,0,270000.0,State servant,Secondary / secondary special,Separated,House / apartment,-19363,-12332,1,0,1,1,Medicine staff,1.0,-18.0,2.0
419,419,M,Y,Y,0,135000.0,Working,Secondary / secondary special,Married,House / apartment,-18820,-3185,1,0,1,0,Low-skill Laborers,2.0,-7.0,1.0
421,421,F,Y,N,0,180000.0,Working,Secondary / secondary special,Married,House / apartment,-10351,-1322,1,0,0,0,NaN,2.0,-16.0,2.0
457,457,F,Y,Y,1,112500.0,Commercial associate,Higher education,Civil marriage,House / apartment,-10551,-3000,1,0,0,1,Core staff,3.0,-17.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26445,26445,F,Y,Y,0,180000.0,Commercial associate,Incomplete higher,Married,House / apartment,-13687,-4144,1,0,0,0,Laborers,2.0,-5.0,0.0
26446,26446,F,N,Y,0,135000.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-16300,-9698,1,0,0,1,Managers,2.0,-41.0,2.0
26447,26447,M,N,Y,2,99000.0,Working,Secondary / secondary special,Married,House / apartment,-14226,-1026,1,1,1,0,Laborers,4.0,-43.0,2.0
26449,26449,F,N,N,0,90000.0,Working,Secondary / secondary special,Married,House / apartment,-10498,-2418,1,1,1,0,NaN,2.0,-2.0,1.0


- 26457행 중 14358행이 중복 데이터

In [ ]:
df['ALL'] = \
    df['child_num'].astype(str) + '_' + df['income_total'].astype(str) + '_' +\
    df['DAYS_BIRTH'].astype(str) + '_' + df['DAYS_EMPLOYED'].astype(str) + '_' +\
    df['work_phone'].astype(str) + '_' + df['phone'].astype(str) + '_' +\
    df['email'].astype(str) + '_' + df['family_size'].astype(str) + '_' +\
    df['gender'].astype(str) + '_' + df['car'].astype(str) + '_' +\
    df['reality'].astype(str) + '_' + df['income_type'].astype(str) + '_' +\
    df['edu_type'].astype(str) + '_' + df['family_type'].astype(str) + '_' +\
    df['house_type'].astype(str) + '_' + df['occyp_type'].astype(str) + '_' +\
    df['credit'].astype(str)

In [ ]:
df = df.sort_values(by='ALL')
num_card = df.groupby('ALL').size()

In [ ]:
df.drop_duplicates(cols, inplace=True)
df['num_card'] = num_card.values
df

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,...,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit,ALL,num_card
8076,8076,F,N,Y,0,101250.0,Working,Secondary / secondary special,Single / not married,House / apartment,...,1,0,1,0,Sales staff,1.0,-7.0,1.0,0_101250.0_-10179_-1813_0_1_0_1.0_F_N_Y_Workin...,1
4803,4803,F,N,Y,0,101250.0,Pensioner,Secondary / secondary special,Widow,House / apartment,...,1,0,1,0,NaN,1.0,-30.0,0.0,0_101250.0_-22059_365243_0_1_0_1.0_F_N_Y_Pensi...,4
12317,12317,F,N,Y,0,101250.0,Pensioner,Secondary / secondary special,Widow,House / apartment,...,1,0,1,0,NaN,1.0,-2.0,1.0,0_101250.0_-22059_365243_0_1_0_1.0_F_N_Y_Pensi...,2
14668,14668,F,N,Y,0,103500.0,Working,Secondary / secondary special,Married,House / apartment,...,1,0,0,1,IT staff,2.0,-5.0,0.0,0_103500.0_-10396_-564_0_0_1_2.0_F_N_Y_Working...,2
8190,8190,F,N,Y,0,103500.0,Working,Secondary / secondary special,Married,House / apartment,...,1,0,0,1,IT staff,2.0,-1.0,1.0,0_103500.0_-10396_-564_0_0_1_2.0_F_N_Y_Working...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2671,2671,F,N,Y,5,157500.0,Working,Secondary / secondary special,Married,House / apartment,...,1,0,0,0,Laborers,7.0,-11.0,2.0,5_157500.0_-13039_-3375_0_0_0_7.0_F_N_Y_Workin...,1
4094,4094,F,N,Y,5,189000.0,Working,Secondary / secondary special,Separated,House / apartment,...,1,0,1,0,NaN,6.0,-50.0,2.0,5_189000.0_-15450_-428_0_1_0_6.0_F_N_Y_Working...,1
10321,10321,M,Y,Y,5,202500.0,Working,Secondary / secondary special,Married,Co-op apartment,...,1,0,0,0,Managers,7.0,-46.0,0.0,5_202500.0_-11384_-2727_0_0_0_7.0_M_Y_Y_Workin...,2
17978,17978,M,Y,Y,5,202500.0,Working,Secondary / secondary special,Married,Co-op apartment,...,1,0,0,0,Managers,7.0,-21.0,1.0,5_202500.0_-11384_-2727_0_0_0_7.0_M_Y_Y_Workin...,2


In [ ]:
df.drop('ALL', axis=1, inplace=True)
df.shape

(12099, 21)

## 2) 변수 제거 및 변환
### 2.1) 변수 제거
- FLAG_MOBIL 모든 값이 1이므로 제거

In [ ]:
df.drop(['FLAG_MOBIL'], axis=1, inplace=True)

### 2.2) 변수 변환

In [ ]:
# 음수값인 변수 절댓값 취해서 양수로 변환

feats = ['DAYS_BIRTH', 'begin_month', 'DAYS_EMPLOYED']
for feat in feats:
    df[feat]=np.abs(df[feat])

## 3) 파생 변수 생성

In [ ]:
# 직관성을 위한 '나이','근속년차', '커리어 시작 나이' 변수

df['Age'] = df['DAYS_BIRTH']//365
df['career_year'] = np.ceil(df['DAYS_EMPLOYED']/365)
df['career_start_age'] = df['Age'] - df['career_year']
df.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,...,phone,email,occyp_type,family_size,begin_month,credit,num_card,Age,career_year,career_start_age
8076,8076,F,N,Y,0,101250.0,Working,Secondary / secondary special,Single / not married,House / apartment,...,1,0,Sales staff,1.0,7.0,1.0,1,27,5.0,22.0
4803,4803,F,N,Y,0,101250.0,Pensioner,Secondary / secondary special,Widow,House / apartment,...,1,0,NaN,1.0,30.0,0.0,4,60,1001.0,-941.0
12317,12317,F,N,Y,0,101250.0,Pensioner,Secondary / secondary special,Widow,House / apartment,...,1,0,NaN,1.0,2.0,1.0,2,60,1001.0,-941.0
14668,14668,F,N,Y,0,103500.0,Working,Secondary / secondary special,Married,House / apartment,...,0,1,IT staff,2.0,5.0,0.0,2,28,2.0,26.0
8190,8190,F,N,Y,0,103500.0,Working,Secondary / secondary special,Married,House / apartment,...,0,1,IT staff,2.0,1.0,1.0,1,28,2.0,26.0


In [ ]:
# ability: 소득 / (살아온 일 수 + 근무일 수) or 소득/근무일 수
df['ability'] = df['income_total'] / (df['DAYS_BIRTH'] + df['DAYS_EMPLOYED'])
df['ability2'] = df['income_total'] / df['DAYS_EMPLOYED']

# income_unit: 소득 / 가족 수
df['income_unit'] = df['income_total']/df['family_size'] # 가족 수 이상치 추후 처리

## 4. 이상치 처리 
- 'income_type' 변수값이 'Pensioner'인 행의 'DAYS_EMPLOYED' 변수값이 대부분 이상치(365243)로 설정되어 있는 문제  
=> 'income_type'이 다른 사람들의 평균으로 대체

In [ ]:
df.loc[:,['income_type', 'DAYS_EMPLOYED']][10:30]

,income_type,DAYS_EMPLOYED
10,Working,-2213
11,Commercial associate,-91
12,Commercial associate,-2162
13,Working,-2474
14,Pensioner,365243
15,Working,-4056
16,Working,-4553
17,Commercial associate,-984
18,Pensioner,365243
19,Working,-6031


In [ ]:
# 연금받는 사람들 나이의 중앙값
a = df[df['income_type']=='Pensioner']['Age'].median()

# 연금받는 사람들이 아닌 사람들 업무 시작 나이의 중앙값
b = df[df['income_type']!='Pensioner']['career_start_age'].median()

# 연금받는 사람들의 근속년수를 (a-b)로 대체
df['career_year'] = np.where(((df['income_type']=='Pensioner') & (df['DAYS_EMPLOYED']==365243)), (a-b), df['career_year'])

# 이에 맞게 'DAYS_EMPLOYED'도 수정
df['DAYS_EMPLOYED'] = np.where(((df['income_type']=='Pensioner') & (df['DAYS_EMPLOYED']==365243)), (a-b)*365, df['DAYS_EMPLOYED'])

df['career_start_age'] = df['Age'] - df['career_year']
df[df['income_type']=='Pensioner'][['DAYS_EMPLOYED', 'career_year', 'career_start_age']]

,DAYS_EMPLOYED,career_year,career_start_age
4803,10585.0,29.0,31.0
12317,10585.0,29.0,31.0
19580,10585.0,29.0,26.0
20605,10585.0,29.0,26.0
4029,10585.0,29.0,28.0
...,...,...,...
24147,10585.0,29.0,6.0
15878,10585.0,29.0,32.0
20752,10585.0,29.0,32.0
22437,10585.0,29.0,35.0


## 5. 결측치 처리

In [ ]:
df.isnull().sum()

index                  0
gender                 0
car                    0
reality                0
child_num              0
income_total           0
income_type            0
edu_type               0
family_type            0
house_type             0
DAYS_BIRTH             0
DAYS_EMPLOYED          0
work_phone             0
phone                  0
email                  0
occyp_type          3666
family_size            0
begin_month            0
credit                 0
num_card               0
Age                    0
career_year            0
career_start_age       0
ability                0
ability2               0
income_unit            0
dtype: int64

In [ ]:
df.occyp_type.value_counts(dropna=False)

NaN                      3666
Laborers                 2158
Sales staff              1182
Core staff               1134
Managers                  989
Drivers                   759
High skill tech staff     457
Accountants               406
Medicine staff            371
Cooking staff             227
Security staff            209
Cleaning staff            191
Private service staff     105
Low-skill Laborers         64
Secretaries                54
Waiters/barmen staff       54
HR staff                   27
IT staff                   25
Realty agents              21
Name: occyp_type, dtype: int64

- 'occyp_type' 변수의 결측치의 비중이 3666/12099, 약 30%로 매우 높음
- 'occyp_type' 변수의 값이 너무 많아 income_total 변수를 기준으로 categorizing하려하였으나 income_total 변수와의 다중공선성 문제 발생  
=> 위와 같은 이유로 'occyp_type' 변수 삭제 결정

In [ ]:
df.drop('occyp_type', axis=1, inplace=True)

## 6. 범주형 변수 처리

In [ ]:
# 범주형 변수 수치형으로 변환

df['gender'] = df['gender'].apply(lambda x: 0 if x=='M' else 1)
df['car'] = df['car'].apply(lambda x: 1 if x=='Y' else 0)
df['reality'] = df['reality'].apply(lambda x: 1 if x=='Y' else 0)

# LabelEncoder
from sklearn.preprocessing import LabelEncoder

features = ['income_type', 'edu_type', 'family_type', 'house_type']
le = LabelEncoder()
df[features] = df[features].apply(le.fit_transform)